<a href="https://colab.research.google.com/github/pneuly/whisper-asr-colab/blob/main/whisper_asr_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import time
import os
import re
import torch
from google.colab import files

!pip install --upgrade git+https://github.com/pneuly/whisper-asr-colab.git
from whisper_asr_colab.docx_generator import DocxGenerator
from whisper_asr_colab.audio import dl_audio, trim_audio
from whisper_asr_colab.utils import write_asr_result, write_diarize_result
from whisper_asr_colab.asr import whisperx_transcribe, faster_whisper_transcribe, realtime_transcribe
from whisper_asr_colab.diarize import diarize

# @title 自動文字起こし（Wisper）{ display-mode: "form" }
# @markdown 以下の設定項目を入力しセルを実行（Ctrl+Enter）<font color="red">※設定項目の説明は下にあります</font>

# @markdown #<b>設定</b>
audiopath = 'https://www.youtube.com/watch?v=xAmEQOqtMvA'  # @param {type:"string"}
model_size = "large-v3" # @param ["large-v3", "large-v2", "large", "medium", "small", "base", "tiny"] {allow-input: true}
diarization = True  # @param {type:"boolean"}
password = None  # @param {type:"string"}
start_time = None  # @param {type:"string"}
end_time = None  # @param {type:"string"}
timestamp_offset = None  # @param {type:"string"}
initial_prompt = "定刻 なりましたので、 です。 ます。"  # @param {type:"string"}
realtime = False  # @param {type:"boolean"}
CHUNK_SIZE = 20
BATCH_SIZE = 16
HUGGING_FACE_TOKEN = ""

# @markdown ###<br/><b>〔設定の説明〕</b>
# @markdown <b>audiopath:</b> 文字起こしする音声ファイルの場所<br/>
# @markdown 　　Youtubeの場合： https://www.youtube.com/......<br/>
# @markdown 　　手動で音声をアップロードした場合： 230401_1010.mp3 など<br/>
# @markdown 　　（アップロード完了まで待って実行してください）<br/>
# @markdown 　　<font color="red">空欄の場合はファイルアップロードボタンが表示されます</font>
# @markdown <br/><b>model_size:</b> 音声認識のモデルサイズ（mediumにすると少し精度が落ちるが早い）
# @markdown <br/><b>diarization:</b> 発言者別の文字起こしファイルを作成するか（Falseにすると早い）
# @markdown #### <br/><b><font color= "blue">以下は必要な場合のみ設定</font></b>
# @markdown <b>password:</b> パスワードを指定（Webexなど）</b>
# @markdown <br/><b>start_time:</b> 開始時間 hh:mm:ss</b>（指定しない場合は最初から）
# @markdown <br/><b>end_time:</b> 終了時間 hh:mm:ss（指定しない場合は最後まで）
# @markdown <br/><b>timestamp_offset:</b> タイムスタンプを指定の時間だけずらす hh:mm:ss（Noneの場合はstart_timeと連動）
# @markdown <br/><b>initial_prompt:</b> キーワード（です。 ます。は句読点を付けるために入れています）
# @markdown <br/><b>reatime: </b><font color="red">ストリーミングをリアルタイムで文字起こしをする場合のみオンにしてください。</font>

# ----- 以下変更不要 ------
if audiopath == "":
    audiopath = list(files.upload())[0]


# --- main ---
# dl audio if needed
dlaudio = False
if not realtime:
    if re.match(r"^(https://).+", audiopath):
        dlaudio = True
        # download and return file name
        audiopath = dl_audio(audiopath, password)
    else:
        # ファイルサイズが小さい場合はアップロード途中の可能性があるためチェックする
        filesize = os.path.getsize(audiopath)
        if filesize < 10 ** 7:  # 10MB未満の場合
            time.sleep(10)
            filesize2 = os.path.getsize(audiopath)
            if (filesize2 - filesize) > 0:
                sys.exit(
                    "ファイルのアップロードが終わっていない可能性があります。アップロード完了後再度実行してください。"
                    )
    # trim
    if start_time or end_time:
        input_path = trim_audio(audiopath, start_time, end_time)
    else:
        input_path = audiopath


# Transcribe
if realtime: # realtime trascription
    realtime_transcribe(audiopath, model_size)
    del model_size
    torch.cuda.empty_cache()
    sys.exit(0)
elif diarization: # use WhisperX
    result = whisperx_transcribe(
        input_path=input_path,
        chunk_size=CHUNK_SIZE,
        batch_size=BATCH_SIZE,
        model_size=model_size,
        initial_prompt=initial_prompt
        )
    segments = result["segments"]
else:  # use faster-whisper
    segments = faster_whisper_transcribe(
        input_path=input_path,
        model_size=model_size,
        initial_prompt=initial_prompt
        )

# write results to text files
outfiles = write_asr_result(
    os.path.basename(input_path),
    segments,
    timestamp_offset
)
for filename in outfiles:
    files.download(filename)

if diarization:
    # Diarize
    filename = write_diarize_result(
        f"{os.path.basename(input_path)}",
        segments,
        timestamp_offset
    )
    files.download(filename)

# gc GPU RAM
del segments
torch.cuda.empty_cache()

# Generate docx
if diarization:
    doc = DocxGenerator()
    doc.txt_to_word(filename)
    files.download(doc.docfilename)

# DL audio file
if dlaudio:
    files.download(audiopath)
